In [ ]:
// spark 初始化，为后续sql的运行准备环境
import $ivy.`com.alibaba:fastjson:1.2.70`
import com.alibaba.fastjson.{JSON, JSONObject}

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.{SparkConf, SparkContext}
var user = sys.env.get("JUPYTERHUB_USER").get
val conf = {
    new SparkConf()
    .setAppName("MTJupyter-Scala-" + user)
    .setMaster("yarn") 
    .set("spark.yarn.queue", "root.zw03.hadoop-waimaiadrd.etl") // spark 的队列，可以根据自己的需要设置
    .set("spark.yarn.archive", "viewfs://hadoop-meituan/user/hadoop-hdp/notebook/spark-jars/spark_libs.zip")
    .set("spark.yarn.stagingDir", "viewfs://hadoop-meituan/ghnn03/sparkstaging")
    .set("spark.executor.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
    .set("spark.driver.extraLibraryPath", "/opt/meituan/nodemanager/lib/native/:/opt/meituan/hadoop/lib/native/")
    .set("spark.executorEnv.JAVA_HOME", "'/usr/local/java18'")
    .set("spark.yarn.appMasterEnv.JAVA_HOME", "'/usr/local/java18'")
    .set("spark.sql.hive.metastorePartitionPruning", "true")
    .set("spark.shuffle.service.enabled", "true")
    .set("spark.dynamicAllocation.enabled", "true")
    .set("spark.dynamicAllocation.minExecutors", "300") //spark的执行Executors
    .set("spark.dynamicAllocation.maxExecutors","600")
    .set("spark.executor.memory","16G")
    .set("spark.driver.memory","7G")
    .set("spark.input.dir.recursive","true")
    .set("spark.sql.ignore.existed.function.enable","true")
    .set("spark.yarn.maxAppAttempts", "2")
    .set("spark.noResourceMaxInterval","86400s")
    .set("spark.scheduler.mode", "FAIR")
    .set("spark.executor.cores","8")
    .set("spark.yarn.executor.memoryOverhead","32024")
    .set("spark.yarn.driver.memoryOverhead","32024")
    .set("spark.hadoop.hive.mt.renew.token.enable", "true")
}
val spark = {
  NotebookSparkSession
    .builder()
    .config(conf)
    .enableHiveSupport.getOrCreate
}
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._

: 

In [ ]:
// 通用udf注册
import scala.collection.mutable.ArrayBuffer
import scala.math
import scala.util.Random;
import java.lang.Math

import org.apache.spark.sql.{DataFrame, Row}
import org.apache.spark.sql.types.{ArrayType, FloatType, DoubleType, IntegerType, LongType, StringType, StructField, StructType}

import scala.collection.immutable.Map
import scala.collection.mutable

def JSONObj2Map(jsonObj: JSONObject): Map[String, String] = {
    var map = mutable.Map[String, String]()
    if(!jsonObj.isEmpty){
        val itr: java.util.Iterator[String] = jsonObj.keySet().iterator()
        while (itr.hasNext) {
            val key = itr.next()
            map += ((key, jsonObj.getString(key)))
        }
    }
    map.toMap
}

import org.apache.hadoop.hive.ql.exec.UDF
import spark.implicits._
val findIndex2 =  (strQuote:String) => {
    val poi = strQuote.split("\073").last
    val ind = strQuote.split("\073").zipWithIndex.filter(_._1 == poi).map(_._2).min
    ind.toString
}
spark.sqlContext.udf.register("findIndex2", findIndex2)
spark.udf.register("findIndex2", findIndex2)

def getDate(delta:Int, current_day:String = "today"):String = {
    val sdf:SimpleDateFormat = new SimpleDateFormat("yyyyMMdd")
    val cal:Calendar = Calendar.getInstance()
    if ( !current_day.equals("today")){
      val dt = sdf.parse(current_day)
      cal.setTime(dt)
    }
    cal.add(Calendar.DATE,delta)
    val day = sdf.format(cal.getTime)
    day
}
def getTimeSuffix():String={
    val dateFormat = new SimpleDateFormat("yyyyMMdd")
    val cla = Calendar.getInstance()
    cla.setTimeInMillis(System.currentTimeMillis())
    val table_suffix = dateFormat.format(cla.getTime)
    table_suffix
}
def dropTable(sqlContext:SQLContext, tableName:String, is_execute_sql:Boolean):Unit={
    var sql = s"Drop table if exists ${tableName}"
    if(is_execute_sql){
        println(s"执行 \n ${sql}")
        sqlContext.sql(sql)
    }
}

def executeSQL(sqlContext:SQLContext, sql:String, is_execute_sql:Boolean):DataFrame={
    println(s"执行 \n ${sql}")
    var result_ = sqlContext.sql(sql)
    result_
}

def genDateList(start: String, end: String): List[String] = {
    import java.util.Date
    import org.joda.time.{DateTime, Instant, Days}
    import org.joda.time.format.{DateTimeFormat, DateTimeFormatter}

    val DATE_KEY_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")
    val DATE_FORMAT: DateTimeFormatter = DateTimeFormat.forPattern("yyyyMMdd")

    val startDate = DATE_KEY_FORMAT.parseDateTime(start).toDateTime
    val endDate = DATE_KEY_FORMAT.parseDateTime(end).toDateTime

    val daysCount = Days.daysBetween(startDate, endDate).getDays() + 1
    (0 until daysCount).map(startDate.plusDays(_)).map(d => d.toString(DATE_FORMAT)).toList
  }


: 

In [ ]:
// 引入相关的库
import java.text.SimpleDateFormat
import java.util.Calendar
import ammonite.ops.ImplicitWd._
import ammonite.ops._
import sys.process._
import java.io._
// var befor1day = "20220411"
// var predict_day = "20220411"

: 

In [ ]:
// 配置区
val is_execute_sql = true
var table_suffix=getTimeSuffix()
// var cur_dt = "20220822"
var cur_dt = 20220907
val dollar = "$"

: 

In [ ]:
// Parameters
val cur_dt = 20221221


: 

In [ ]:
// 同期4周数据
def get_his_charge_nt(sqlContext: SQLContext, cur_day:String) = {
  val before1day = getDate(-1, cur_day)
  val before1week = getDate(-7, cur_day)
  val before2week = getDate(-14, cur_day)
  val before3week = getDate(-21, cur_day)
  val before4week = getDate(-28, cur_day)
  val table_name_nt = s"mart_waimaiad.historical_contemporaneous_charge_data"
  // 创建
  // dropTable(spark.sqlContext, table_name_nt, is_execute_sql)
  var create_table_nt =
    s"""
    CREATE TABLE IF NOT EXISTS ${table_name_nt}
    (
        poi_id string,
        -- prod string,

        impr map<int, double> comment '前一天四渠道impr',
        click map<int, double> comment '前一天四渠道点击',
        charge map<int, double> comment '前一天四渠道广告原价收入',
        cpc map<int, double> comment '前一天四渠道cpc',
        revenue map<int, double> comment '前一天四渠道gmv',
        order_num map<int, double> comment '前一天四渠道单数',
        mt_butie map<int, double> comment '前一天四渠道美团补贴',

        impr_before1week map<int, double> comment      '前一周四渠道impr',
        click_before1week map<int, double> comment     '前一周四渠道点击',
        charge_before1week map<int, double> comment    '前一周四渠道广告原价收入',
        cpc_before1week map<int, double> comment       '前一周四渠道cpc',
        revenue_before1week map<int, double> comment   '前一周四渠道gmv',
        order_num_before1week map<int, double> comment '前一周四渠道单数',
        mt_butie_before1week map<int, double> comment  '前一周四渠道美团补贴',

        impr_before2week map<int, double> comment '前2周四渠道impr',
        click_before2week map<int, double> comment '前2周四渠道点击',
        charge_before2week map<int, double> comment '前2周四渠道广告原价收入',
        cpc_before2week map<int, double> comment '前2周四渠道cpc',
        revenue_before2week map<int, double> comment '前2周四渠道gmv',
        order_num_before2week map<int, double> comment '前2周四渠道单数',
        mt_butie_before2week map<int, double> comment '前2周四渠道美团补贴',

        impr_before3week map<int, double> comment '前3周四渠道impr',
        click_before3week map<int, double> comment '前3周四渠道点击',
        charge_before3week map<int, double> comment '前3周四渠道广告原价收入',
        cpc_before3week map<int, double> comment '前3周四渠道cpc',
        revenue_before3week map<int, double> comment '前3周四渠道gmv',
        order_num_before3week map<int, double> comment '前3周四渠道单数',
        mt_butie_before3week map<int, double> comment '前3周四渠道美团补贴',

        impr_before4week map<int, double> comment '前4周四渠道impr',
        click_before4week map<int, double> comment '前4周四渠道点击',
        charge_before4week map<int, double> comment '前4周四渠道广告原价收入',
        cpc_before4week map<int, double> comment '前4周四渠道cpc',
        revenue_before4week map<int, double> comment '前4周四渠道gmv',
        order_num_before4week map<int, double> comment '前4周四渠道单数',
        mt_butie_before4week map<int, double> comment '前4周四渠道美团补贴'
    )
    COMMENT ''
    PARTITIONED BY (dt string)

    STORED AS ORC
    """.stripMargin
  executeSQL(spark.sqlContext, create_table_nt, is_execute_sql)

  // 写入
  spark.sql("set spark.sql.shuffle.partitions=2000")
  val dollar = "$"
  var hive_sql_nt =
      s"""
        INSERT OVERWRITE TABLE ${table_name_nt} PARTITION (dt='${cur_day}')
          select poi_id,
          map(1,sum(coalesce(impr_dj,0)),2,sum(coalesce(impr_bj,0)),3,sum(coalesce(impr_ss,0)),4,sum(coalesce(impr_lkb,0))) as impr,
                  map(1,sum(coalesce(clk_dj,0)),2,sum(coalesce(clk_bj,0)),3,sum(coalesce(clk_ss,0)),4,sum(coalesce(clk_lkb,0))) as clk,
                  map(1,sum(coalesce(charge_dj,0)),2,sum(coalesce(charge_bj,0)),3,sum(coalesce(charge_ss,0)),4,sum(coalesce(charge_lkb,0))) as charge,
                  map(1,sum(coalesce(cpc_dj,0)),2,sum(coalesce(cpc_bj,0)),3,sum(coalesce(cpc_ss,0)),4,sum(coalesce(cpc_lkb,0))) as cpc,
                  map(1,sum(coalesce(revenue_dj,0)),2,sum(coalesce(revenue_bj,0)),3,sum(coalesce(revenue_ss,0)),4,sum(coalesce(revenue_lkb,0))) as revenue,
                  map(1,sum(coalesce(order_num_dj,0)),2,sum(coalesce(order_num_bj,0)),3,sum(coalesce(order_num_ss,0)),4,sum(coalesce(order_num_lkb,0))) as order_num,
                  map(1,sum(coalesce(mt_butie_dj,0)),2,sum(coalesce(mt_butie_bj,0)),3,sum(coalesce(mt_butie_ss,0)),4,sum(coalesce(mt_butie_lkb,0))) as mt_butie,

                  map(1,sum(coalesce(impr_before1week_dj,0)),2,sum(coalesce(impr_before1week_bj,0)),3,sum(coalesce(impr_before1week_ss,0)),4,sum(coalesce(impr_before1week_lkb,0))) as impr_before1week,
                  map(1,sum(coalesce(clk_before1week_dj,0)),2,sum(coalesce(clk_before1week_bj,0)),3,sum(coalesce(clk_before1week_ss,0)),4,sum(coalesce(clk_before1week_lkb,0))) as clk_before1week,
                  map(1,sum(coalesce(charge_before1week_dj,0)),2,sum(coalesce(charge_before1week_bj,0)),3,sum(coalesce(charge_before1week_ss,0)),4,sum(coalesce(charge_before1week_lkb,0))) as charge_before1week,
                  map(1,sum(coalesce(cpc_before1week_dj,0)),2,sum(coalesce(cpc_before1week_bj,0)),3,sum(coalesce(cpc_before1week_ss,0)),4,sum(coalesce(cpc_before1week_lkb,0))) as cpc_before1week,
                  map(1,sum(coalesce(revenue_before1week_dj,0)),2,sum(coalesce(revenue_before1week_bj,0)),3,sum(coalesce(revenue_before1week_ss,0)),4,sum(coalesce(revenue_before1week_lkb,0))) as revenue_before1week,
                  map(1,sum(coalesce(order_num_before1week_dj,0)),2,sum(coalesce(order_num_before1week_bj,0)),3,sum(coalesce(order_num_before1week_ss,0)),4,sum(coalesce(order_num_before1week_lkb,0))) as order_num_before1week,
                  map(1,sum(coalesce(mt_butie_before1week_dj,0)),2,sum(coalesce(mt_butie_before1week_bj,0)),3,sum(coalesce(mt_butie_before1week_ss,0)),4,sum(coalesce(mt_butie_before1week_lkb,0))) as mt_butie_before1week,

                  map(1,sum(coalesce(impr_before2week_dj,0)),2,sum(coalesce(impr_before2week_bj,0)),3,sum(coalesce(impr_before2week_ss,0)),4,sum(coalesce(impr_before2week_lkb,0))) as impr_before2week,
                  map(1,sum(coalesce(clk_before2week_dj,0)),2,sum(coalesce(clk_before2week_bj,0)),3,sum(coalesce(clk_before2week_ss,0)),4,sum(coalesce(clk_before2week_lkb,0))) as clk_before2week,
                  map(1,sum(coalesce(charge_before2week_dj,0)),2,sum(coalesce(charge_before2week_bj,0)),3,sum(coalesce(charge_before2week_ss,0)),4,sum(coalesce(charge_before2week_lkb,0))) as charge_before2week,
                  map(1,sum(coalesce(cpc_before2week_dj,0)),2,sum(coalesce(cpc_before2week_bj,0)),3,sum(coalesce(cpc_before2week_ss,0)),4,sum(coalesce(cpc_before2week_lkb,0))) as cpc_before2week,
                  map(1,sum(coalesce(revenue_before2week_dj,0)),2,sum(coalesce(revenue_before2week_bj,0)),3,sum(coalesce(revenue_before2week_ss,0)),4,sum(coalesce(revenue_before2week_lkb,0))) as revenue_before2week,
                  map(1,sum(coalesce(order_num_before2week_dj,0)),2,sum(coalesce(order_num_before2week_bj,0)),3,sum(coalesce(order_num_before2week_ss,0)),4,sum(coalesce(order_num_before2week_lkb,0))) as order_num_before2week,
                  map(1,sum(coalesce(mt_butie_before2week_dj,0)),2,sum(coalesce(mt_butie_before2week_bj,0)),3,sum(coalesce(mt_butie_before2week_ss,0)),4,sum(coalesce(mt_butie_before2week_lkb,0))) as mt_butie_before2week,

                  map(1,sum(coalesce(impr_before3week_dj,0)),2,sum(coalesce(impr_before3week_bj,0)),3,sum(coalesce(impr_before3week_ss,0)),4,sum(coalesce(impr_before3week_lkb,0))) as impr_before3week,
                  map(1,sum(coalesce(clk_before3week_dj,0)),2,sum(coalesce(clk_before3week_bj,0)),3,sum(coalesce(clk_before3week_ss,0)),4,sum(coalesce(clk_before3week_lkb,0))) as clk_before3week,
                  map(1,sum(coalesce(charge_before3week_dj,0)),2,sum(coalesce(charge_before3week_bj,0)),3,sum(coalesce(charge_before3week_ss,0)),4,sum(coalesce(charge_before3week_lkb,0))) as charge_before3week,
                  map(1,sum(coalesce(cpc_before3week_dj,0)),2,sum(coalesce(cpc_before3week_bj,0)),3,sum(coalesce(cpc_before3week_ss,0)),4,sum(coalesce(cpc_before3week_lkb,0))) as cpc_before3week,
                  map(1,sum(coalesce(revenue_before3week_dj,0)),2,sum(coalesce(revenue_before3week_bj,0)),3,sum(coalesce(revenue_before3week_ss,0)),4,sum(coalesce(revenue_before3week_lkb,0))) as revenue_before3week,
                  map(1,sum(coalesce(order_num_before3week_dj,0)),2,sum(coalesce(order_num_before3week_bj,0)),3,sum(coalesce(order_num_before3week_ss,0)),4,sum(coalesce(order_num_before3week_lkb,0))) as order_num_before3week,
                  map(1,sum(coalesce(mt_butie_before3week_dj,0)),2,sum(coalesce(mt_butie_before3week_bj,0)),3,sum(coalesce(mt_butie_before3week_ss,0)),4,sum(coalesce(mt_butie_before3week_lkb,0))) as mt_butie_before3week,

                  map(1,sum(coalesce(impr_before4week_dj,0)),2,sum(coalesce(impr_before4week_bj,0)),3,sum(coalesce(impr_before4week_ss,0)),4,sum(coalesce(impr_before4week_lkb,0))) as impr_before4week,
                  map(1,sum(coalesce(clk_before4week_dj,0)),2,sum(coalesce(clk_before4week_bj,0)),3,sum(coalesce(clk_before4week_ss,0)),4,sum(coalesce(clk_before4week_lkb,0))) as clk_before4week,
                  map(1,sum(coalesce(charge_before4week_dj,0)),2,sum(coalesce(charge_before4week_bj,0)),3,sum(coalesce(charge_before4week_ss,0)),4,sum(coalesce(charge_before4week_lkb,0))) as charge_before4week,
                  map(1,sum(coalesce(cpc_before4week_dj,0)),2,sum(coalesce(cpc_before4week_bj,0)),3,sum(coalesce(cpc_before4week_ss,0)),4,sum(coalesce(cpc_before4week_lkb,0))) as cpc_before4week,
                  map(1,sum(coalesce(revenue_before4week_dj,0)),2,sum(coalesce(revenue_before4week_bj,0)),3,sum(coalesce(revenue_before4week_ss,0)),4,sum(coalesce(revenue_before4week_lkb,0))) as revenue_before4week,
                  map(1,sum(coalesce(order_num_before4week_dj,0)),2,sum(coalesce(order_num_before4week_bj,0)),3,sum(coalesce(order_num_before4week_ss,0)),4,sum(coalesce(order_num_before4week_lkb,0))) as order_num_before4week,
                  map(1,sum(coalesce(mt_butie_before4week_dj,0)),2,sum(coalesce(mt_butie_before4week_bj,0)),3,sum(coalesce(mt_butie_before4week_ss,0)),4,sum(coalesce(mt_butie_before4week_lkb,0))) as mt_butie_before4week
                  from(
                SELECT coalesce(order_log.poi_id, charge_log.poi_id) as poi_id,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',impr, 0) AS impr_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',impr, 0) AS impr_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',impr, 0) AS impr_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',impr, 0) AS impr_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',impr_before1week, 0) AS impr_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',impr_before1week, 0) AS impr_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',impr_before1week, 0) AS impr_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',impr_before1week, 0) AS impr_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',impr_before2week, 0) AS impr_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',impr_before2week, 0) AS impr_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',impr_before2week, 0) AS impr_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',impr_before2week, 0) AS impr_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',impr_before3week, 0) AS impr_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',impr_before3week, 0) AS impr_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',impr_before3week, 0) AS impr_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',impr_before3week, 0) AS impr_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',impr_before4week, 0) AS impr_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',impr_before4week, 0) AS impr_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',impr_before4week, 0) AS impr_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',impr_before4week, 0) AS impr_before4week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',clk, 0) AS clk_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',clk, 0) AS clk_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',clk, 0) AS clk_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',clk, 0) AS clk_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',clk_before1week, 0) AS clk_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',clk_before1week, 0) AS clk_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',clk_before1week, 0) AS clk_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',clk_before1week, 0) AS clk_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',clk_before2week, 0) AS clk_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',clk_before2week, 0) AS clk_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',clk_before2week, 0) AS clk_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',clk_before2week, 0) AS clk_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',clk_before3week, 0) AS clk_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',clk_before3week, 0) AS clk_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',clk_before3week, 0) AS clk_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',clk_before3week, 0) AS clk_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',clk_before4week, 0) AS clk_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',clk_before4week, 0) AS clk_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',clk_before4week, 0) AS clk_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',clk_before4week, 0) AS clk_before4week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',charge, 0) AS charge_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',charge, 0) AS charge_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',charge, 0) AS charge_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',charge, 0) AS charge_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',charge_before1week, 0) AS charge_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',charge_before1week, 0) AS charge_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',charge_before1week, 0) AS charge_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',charge_before1week, 0) AS charge_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',charge_before2week, 0) AS charge_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',charge_before2week, 0) AS charge_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',charge_before2week, 0) AS charge_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',charge_before2week, 0) AS charge_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',charge_before3week, 0) AS charge_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',charge_before3week, 0) AS charge_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',charge_before3week, 0) AS charge_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',charge_before3week, 0) AS charge_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',charge_before4week, 0) AS charge_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',charge_before4week, 0) AS charge_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',charge_before4week, 0) AS charge_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',charge_before4week, 0) AS charge_before4week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',cpc, 0) AS cpc_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',cpc, 0) AS cpc_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',cpc, 0) AS cpc_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',cpc, 0) AS cpc_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',cpc_before1week, 0) AS cpc_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',cpc_before1week, 0) AS cpc_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',cpc_before1week, 0) AS cpc_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',cpc_before1week, 0) AS cpc_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',cpc_before2week, 0) AS cpc_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',cpc_before2week, 0) AS cpc_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',cpc_before2week, 0) AS cpc_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',cpc_before2week, 0) AS cpc_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',cpc_before3week, 0) AS cpc_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',cpc_before3week, 0) AS cpc_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',cpc_before3week, 0) AS cpc_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',cpc_before3week, 0) AS cpc_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',cpc_before4week, 0) AS cpc_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',cpc_before4week, 0) AS cpc_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',cpc_before4week, 0) AS cpc_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',cpc_before4week, 0) AS cpc_before4week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',revenue, 0) AS revenue_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',revenue, 0) AS revenue_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',revenue, 0) AS revenue_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',revenue, 0) AS revenue_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',revenue_before1week, 0) AS revenue_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',revenue_before1week, 0) AS revenue_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',revenue_before1week, 0) AS revenue_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',revenue_before1week, 0) AS revenue_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',revenue_before2week, 0) AS revenue_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',revenue_before2week, 0) AS revenue_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',revenue_before2week, 0) AS revenue_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',revenue_before2week, 0) AS revenue_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',revenue_before3week, 0) AS revenue_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',revenue_before3week, 0) AS revenue_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',revenue_before3week, 0) AS revenue_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',revenue_before3week, 0) AS revenue_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',revenue_before4week, 0) AS revenue_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',revenue_before4week, 0) AS revenue_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',revenue_before4week, 0) AS revenue_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',revenue_before4week, 0) AS revenue_before4week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',order_num, 0) AS order_num_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',order_num, 0) AS order_num_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',order_num, 0) AS order_num_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',order_num, 0) AS order_num_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',order_num_before1week, 0) AS order_num_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',order_num_before1week, 0) AS order_num_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',order_num_before1week, 0) AS order_num_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',order_num_before1week, 0) AS order_num_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',order_num_before2week, 0) AS order_num_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',order_num_before2week, 0) AS order_num_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',order_num_before2week, 0) AS order_num_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',order_num_before2week, 0) AS order_num_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',order_num_before3week, 0) AS order_num_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',order_num_before3week, 0) AS order_num_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',order_num_before3week, 0) AS order_num_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',order_num_before3week, 0) AS order_num_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',order_num_before4week, 0) AS order_num_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',order_num_before4week, 0) AS order_num_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',order_num_before4week, 0) AS order_num_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',order_num_before4week, 0) AS order_num_before4week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',mt_butie, 0) AS mt_butie_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',mt_butie, 0) AS mt_butie_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',mt_butie, 0) AS mt_butie_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',mt_butie, 0) AS mt_butie_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',mt_butie_before1week, 0) AS mt_butie_before1week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',mt_butie_before1week, 0) AS mt_butie_before1week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',mt_butie_before1week, 0) AS mt_butie_before1week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',mt_butie_before1week, 0) AS mt_butie_before1week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',mt_butie_before2week, 0) AS mt_butie_before2week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',mt_butie_before2week, 0) AS mt_butie_before2week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',mt_butie_before2week, 0) AS mt_butie_before2week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',mt_butie_before2week, 0) AS mt_butie_before2week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',mt_butie_before3week, 0) AS mt_butie_before3week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',mt_butie_before3week, 0) AS mt_butie_before3week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',mt_butie_before3week, 0) AS mt_butie_before3week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',mt_butie_before3week, 0) AS mt_butie_before3week_lkb,
                      if(coalesce(order_log.prod, charge_log.prod) = '1',mt_butie_before4week, 0) AS mt_butie_before4week_dj,
                      if(coalesce(order_log.prod, charge_log.prod) = '2',mt_butie_before4week, 0) AS mt_butie_before4week_bj,
                      if(coalesce(order_log.prod, charge_log.prod) = '3',mt_butie_before4week, 0) AS mt_butie_before4week_ss,
                      if(coalesce(order_log.prod, charge_log.prod) = '4',mt_butie_before4week, 0) AS mt_butie_before4week_lkb
                       from(
                        select dt, -- pv_id,
                               poi_id,
                               prod,
                               cast(sum(if(dt=${before1day},if(act=3, 1, 0),0)) as double) as impr,
                              cast(sum(if(dt=${before1day},if(act=2, 1, 0),0)) as double) as clk,
                              cast(sum(if(dt=${before1day},if(is_charge=1, ad_actl_inc, 0),0)) as double) as charge,
                              cast(if(sum(if(dt=${before1day},if(act=2, 1, 0),0))>0, sum(if(dt=${before1day},if(is_charge=1, ad_actl_inc, 0),0))/sum(if(dt=${before1day},if(act=2, 1, 0),0)),0) as double) as cpc,
                              cast(sum(if(dt=${before1week},if(act=3, 1, 0),0)) as double) as impr_before1week,
                              cast(sum(if(dt=${before1week},if(act=2, 1, 0),0)) as double) as clk_before1week,
                              cast(sum(if(dt=${before1week},if(is_charge=1, ad_actl_inc, 0),0)) as double) as charge_before1week,
                              cast(if(sum(if(dt=${before1week},if(act=2, 1, 0),0))>0, sum(if(dt=${before1week},if(is_charge=1, ad_actl_inc, 0),0))/sum(if(dt=${before1week},if(act=2, 1, 0),0)),0) as double) as cpc_before1week,
                              cast(sum(if(dt=${before2week},if(act=3, 1, 0),0)) as double) as impr_before2week,
                              cast(sum(if(dt=${before2week},if(act=2, 1, 0),0)) as double) as clk_before2week,
                              cast(sum(if(dt=${before2week},if(is_charge=1, ad_actl_inc, 0),0)) as double) as charge_before2week,
                              cast(if(sum(if(dt=${before2week},if(act=2, 1, 0),0))>0, sum(if(dt=${before2week},if(is_charge=1, ad_actl_inc, 0),0))/sum(if(dt=${before2week},if(act=2, 1, 0),0)),0) as double) as cpc_before2week,
                              cast(sum(if(dt=${before3week},if(act=3, 1, 0),0)) as double) as impr_before3week,
                              cast(sum(if(dt=${before3week},if(act=2, 1, 0),0)) as double) as clk_before3week,
                              cast(sum(if(dt=${before3week},if(is_charge=1, ad_actl_inc, 0),0)) as double) as charge_before3week,
                              cast(if(sum(if(dt=${before3week},if(act=2, 1, 0),0))>0, sum(if(dt=${before3week},if(is_charge=1, ad_actl_inc, 0),0))/sum(if(dt=${before3week},if(act=2, 1, 0),0)),0) as double) as cpc_before3week,
                              cast(sum(if(dt=${before4week},if(act=3, 1, 0),0)) as double) as impr_before4week,
                              cast(sum(if(dt=${before4week},if(act=2, 1, 0),0)) as double) as clk_before4week,
                              cast(sum(if(dt=${before4week},if(is_charge=1, ad_actl_inc, 0),0)) as double) as charge_before4week,
                              cast(if(sum(if(dt=${before4week},if(act=2, 1, 0),0))>0, sum(if(dt=${before4week},if(is_charge=1, ad_actl_inc, 0),0))/sum(if(dt=${before4week},if(act=2, 1, 0),0)),0) as double) as cpc_before4week
                                                  from (
                                SELECT dt,
                                       poi_id,
                                       ad_request_id AS pv_id,
                                       CASE prod_type
                                            WHEN 'yzs2bj_cpm' THEN '2'
                                            WHEN 'yzs2tj_cpm' THEN '4'
                                            WHEN 'yzs2lb_cpc' THEN '1'
                                            WHEN 'yzs2ss_cpc' THEN '3'
                                            ELSE 'none'
                                             END AS prod,
                                    max(is_charge) as is_charge,
                                    max(ad_actl_inc) as ad_actl_inc,
                                    act
                                  FROM mart_waimai_dw_ad.fact_flow_ad_entry_mv
                                 WHERE dt in (${before1day}, ${before1week},${before2week},${before3week},${before4week})
                                   AND is_valid='PASS'
                                   AND prod_type IN ('yzs2lb_cpc', 'yzs2ss_cpc', 'yzs2tj_cpm', 'yzs2bj_cpm')
                                   group by dt,poi_id,ad_request_id,prod_type,act
                               ) t_charge
                         group by dt,
                                  poi_id,
                                  prod
                       ) order_log
                  FULL JOIN(
                        select dt, -- pv_id,
         poi_id,
                               prod,
                           cast(sum(if(dt=${before1day},revenue,0)) as double) as revenue,
                          cast(sum(if(dt=${before1day},order_num,0)) as double) as order_num,
                          cast(sum(if(dt=${before1day},mt_butie,0)) as double) as mt_butie,
                          cast(sum(if(dt=${before1week},revenue,0)) as double) as revenue_before1week,
                          cast(sum(if(dt=${before1week},order_num,0)) as double) as order_num_before1week,
                          cast(sum(if(dt=${before1week},mt_butie,0)) as double) as mt_butie_before1week,
                          cast(sum(if(dt=${before2week},revenue,0)) as double) as revenue_before2week,
                          cast(sum(if(dt=${before2week},order_num,0)) as double) as order_num_before2week,
                          cast(sum(if(dt=${before2week},mt_butie,0)) as double) as mt_butie_before2week,
                          cast(sum(if(dt=${before3week},revenue,0)) as double) as revenue_before3week,
                          cast(sum(if(dt=${before3week},order_num,0)) as double) as order_num_before3week,
                          cast(sum(if(dt=${before3week},mt_butie,0)) as double) as mt_butie_before3week,
                          cast(sum(if(dt=${before4week},revenue,0)) as double) as revenue_before4week,
                          cast(sum(if(dt=${before4week},order_num,0)) as double) as order_num_before4week,
                          cast(sum(if(dt=${before4week},mt_butie,0)) as double) as mt_butie_before4week
                          from (
                                select dt,
                                       ad_request_id pv_id,
                                       poi_id,
                                       CASE prod_type
                                            WHEN 'yzs2bj_cpm' THEN '2'
                                            WHEN 'yzs2tj_cpm' THEN '4'
                                            WHEN 'yzs2lb_cpc' THEN '1'
                                            WHEN 'yzs2ss_cpc' THEN '3'
                                            ELSE 'none'
                                             END AS prod,
                                       max(sub_total) as revenue,
                                       max(sub_ord_num) as order_num,
                                       max(sub_mt_charge_fee) as mt_butie
                                  from mart_waimai_dw_ad.fact_flow_ad_sdk_entry_order_view
                                 where dt in (${before1day}, ${before1week},${before2week},${before3week},${before4week})
                                   and is_ad=1
                                   and prod_type IN ('yzs2lb_cpc', 'yzs2ss_cpc', 'yzs2tj_cpm', 'yzs2bj_cpm')
                                   group by dt,poi_id,ad_request_id,prod_type
                               ) t_order
                         group by dt,
                                  poi_id,
                                  prod
                       ) charge_log
                    on order_log.dt = charge_log.dt
                   and order_log.poi_id = charge_log.poi_id
                   and order_log.prod = charge_log.prod
               )
               group by poi_id
      """.stripMargin
  print(hive_sql_nt)
  executeSQL(spark.sqlContext, hive_sql_nt, is_execute_sql)
}



: 

In [ ]:
get_his_charge_nt(spark.sqlContext, cur_dt.toString)

: 

: 